# Preapre

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\r3080\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\r3080\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import wrds
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
pd.set_option('display.max_columns', None)
import statsmodels.api as sm
from statsmodels.api import add_constant
from sklearn.metrics import r2_score
from CompanyTextSentimentAnalyzer import TextSentimentAnalyser


import warnings
warnings.filterwarnings("ignore")

In [3]:
db = wrds.Connection(wrds_username = 'rotmanren') # Change to user's username


Loading library list...
Done


# Data injection

In [4]:
# Load all data
startYear = 2009
# Accruals need data from 2009 to obtain changes
endYear = 2022
dataquery1 = f"""
            SELECT a.DATADATE, a.cik, a.GVKEY, a.tic, a.fyear, a.ib, a.spi, a.at, a.dvc, a.act, a.che, a.lct, a.dlc, a.ni, a.txp, a.revt, a.dp, a.csho, a.ceq, a.ivao, a.lt, a.ajex, a.dltt, a.ivst, a.pstk, b.cshom, b.prccm
            FROM comp.funda as a
            JOIN comp_na_daily_all.secm as b
            ON a.tic = b.tic AND a.fyear = b.cyear
            WHERE consol = 'C'
            AND popsrc = 'D'
            AND indfmt = 'INDL'
            AND datafmt = 'STD' 
            AND fyear BETWEEN '{startYear}' AND '{endYear}'
            AND b.cmth = 6.0
        """
inforaw = db.raw_sql(dataquery1)
# Fill NA for selected variables 
cols = ['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']
inforaw[cols] = inforaw[cols].fillna(0)
inforaw = inforaw.dropna()
inforaw = inforaw.reset_index()
inforaw = inforaw.sort_values(by=['tic', 'fyear'])

# Adjust for stock split
inforaw['csho'] = inforaw['csho'] * inforaw['ajex']
inforaw['mrk_eq'] = inforaw['cshom'] * inforaw['prccm']
inforaw

,index,datadate,cik,gvkey,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,ni,txp,revt,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq
12808,15397,2009-06-30,0000770034,012837,3ADTI,2009.0,-5.586,0.000,5.747,0.0,3.990,1.320,8.328,7.241,-5.586,0.0,0.000,0.490,32.01500,-2.790,0.000,8.360,1.00,0.021,0.000,0.177,26198000.0,0.0600,1571880.0
22947,36157,2009-12-31,0000749254,024792,3AEMI,2009.0,-6.623,0.000,26.375,0.0,10.488,6.570,12.422,1.092,-6.623,0.0,29.422,1.610,22.08600,-11.200,0.000,37.575,1.00,18.406,0.800,0.000,21786000.0,0.3000,6535800.0
22948,36158,2010-12-31,0000749254,024792,3AEMI,2010.0,-1.305,-1.435,22.149,0.0,7.138,3.022,13.738,1.131,-1.305,0.0,27.493,1.801,33.14600,-9.766,0.000,31.915,1.00,17.590,0.601,0.000,22279000.0,0.2800,6238120.0
22922,36096,2009-12-31,0000879181,024668,3AHOM,2009.0,-13.098,0.000,239.297,0.0,66.048,22.194,264.160,229.120,-13.098,0.0,241.540,25.690,17.57300,-37.411,8.352,276.276,1.00,0.003,0.250,0.000,17573000.0,0.2500,4393250.0
367,512,2011-12-31,0000276750,001367,3AMBEQ,2011.0,-0.080,0.000,0.906,0.0,0.906,0.906,0.022,0.000,-0.080,0.0,0.000,0.000,4.66600,0.884,0.000,0.022,1.00,0.000,0.000,0.000,4666000.0,0.2550,1189830.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58680,105186,2018-02-28,0001489353,178186,ZZZOF,2017.0,-6.558,0.000,27.923,0.0,2.078,1.071,4.458,0.017,-6.558,0.0,0.000,0.010,0.99482,23.400,0.808,4.523,0.01,0.065,0.546,0.000,80651000.0,0.3399,27413274.9
58681,105187,2019-02-28,0001489353,178186,ZZZOF,2018.0,-26.442,-0.029,6.954,0.0,0.234,0.152,2.956,0.559,-26.442,0.0,0.000,0.015,1.22361,3.998,0.000,2.956,0.01,0.000,0.139,0.000,99482000.0,0.1692,16832354.4
58682,105188,2020-02-29,0001489353,178186,ZZZOF,2019.0,-7.685,0.002,1.720,0.0,0.086,0.000,3.726,0.841,-7.685,0.0,0.000,0.006,1.93957,-2.006,0.000,3.726,0.01,0.000,0.000,0.000,122527000.0,0.0140,1715378.0
58683,105189,2021-02-28,0001489353,178186,ZZZOF,2020.0,-0.521,0.000,0.034,0.0,0.023,0.013,1.195,0.157,0.610,0.0,0.000,0.005,1.93957,-1.161,0.000,1.195,0.01,0.000,0.000,0.000,193957000.0,0.0140,2715398.0


In [5]:
# Load analyst data for three years, training
startYear_an = 2010
endYear_an = 2018
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_one_raw_train = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_one_raw_train = df_analyst_one_raw_train.dropna()
df_analyst_one_raw_train = df_analyst_one_raw_train.reset_index()


startYear_an = 2010
endYear_an = 2017
dataquery_two = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '2'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_two)
df_analyst_two_raw_train = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_two_raw_train = df_analyst_two_raw_train.dropna()
df_analyst_two_raw_train = df_analyst_two_raw_train.reset_index()

startYear_an = 2010
endYear_an = 2016
dataquery_three = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '3'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_three)
df_analyst_three_raw_train = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_three_raw_train = df_analyst_three_raw_train.dropna()
df_analyst_three_raw_train = df_analyst_three_raw_train.reset_index()


In [6]:
df_analyst_one_raw_train

,index,ticker,fpedats,statpers,actual,medest,gvkey,TICKER
0,0,0000,2014-12-31,2014-06-19,1.2100,0.58,192458,0000
1,1,0000,2015-12-31,2015-06-18,1.0200,0.83,192458,0000
2,3,0001,2015-12-31,2015-06-18,0.7800,0.60,18660,0001
3,4,0001,2018-12-31,2018-06-14,-0.2500,-0.05,18660,0001
4,5,0001,2016-12-31,2016-06-16,0.6200,0.79,18660,0001
...,...,...,...,...,...,...,...,...
32321,34790,XTLB,2015-12-31,2015-06-18,-0.0850,-0.10,241335,XTLB
32322,34791,YOGA,2018-12-31,2018-06-14,-0.8800,-0.83,32192,YOGA
32323,34793,ZOS,2013-09-30,2013-06-20,-4.1282,3.88,11670,ZOS
32324,34794,ZTO,2017-12-31,2017-06-15,4.4400,4.18,28423,ZTO


In [7]:
# Load analyst data for three years, testing
startYear_an = 2019
endYear_an = 2019
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_raw_test_2019 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.dropna()
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.reset_index()

startYear_an = 2022
endYear_an = 2022
dataquery_two = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_two)
df_analyst_raw_test_2022 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.dropna()
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.reset_index()


In [8]:
pd.merge(inforaw, df_analyst_raw_test_2019, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])


,index_x,datadate,cik,gvkey_x,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,ni,txp,revt,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq,index_y,ticker,fpedats,statpers,actual,medest,fpi,gvkey_y,TICKER
0,25121,2019-12-31,0001158114,018564,AAOI,2019.0,-66.049,-11.030,466.825,0.000,192.804,67.028,91.355,40.663,-66.049,0.000,190.872,24.014,20.140,273.794,0.000,193.031,1.0,101.676,7.051,0.0,1.994000e+07,10.28,2.049832e+08,62,AAOI,2019-12-31,2019-06-20,-0.8600,-1.07,1,18564,AAOI
1,32397,2019-12-31,0000824142,021542,AAON,2019.0,53.711,0.000,371.424,16.645,187.549,44.373,56.028,0.000,53.711,0.000,469.333,22.766,52.079,290.140,0.597,0.000,1.0,0.000,17.576,0.0,5.211500e+07,50.18,2.615131e+09,1417,AAON,2019-12-31,2019-06-20,1.0200,1.20,1,21542,AAON
2,769,2019-09-30,0000320193,001690,AAPL,2019.0,55256.000,0.000,338516.000,14129.000,162819.000,100580.000,105718.000,16240.000,55256.000,0.000,260174.000,11300.000,17772.944,90488.000,105341.000,248028.000,4.0,91807.000,51736.000,0.0,4.601075e+09,197.92,9.106448e+11,1213,AAPL,2019-09-30,2019-06-20,2.9725,2.86,1,1690,AAPL
3,20899,2019-12-31,0001551152,016101,ABBV,2019.0,7882.000,-4275.000,89115.000,6533.000,49519.000,39924.000,15585.000,3862.000,7882.000,0.000,33266.000,2017.000,1478.912,-8172.000,93.000,97287.000,1.0,63226.000,0.000,0.0,1.478365e+09,72.72,1.075067e+11,63,ABBV,2019-12-31,2019-06-20,8.9400,8.82,1,16101,ABBV
4,681,2019-10-31,0000771497,001410,ABM,2019.0,127.500,-17.600,3692.600,47.700,1275.400,58.500,902.400,57.200,127.400,3.500,6498.600,107.400,66.571,1542.000,0.000,2150.600,1.0,744.200,0.000,0.0,6.635800e+07,40.00,2.654320e+09,273,ABM,2019-10-31,2019-06-20,2.0500,2.03,1,1410,ABM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,52378,2019-07-31,0001713683,033037,ZS,2019.0,-28.655,-13.379,604.162,0.000,496.086,364.646,261.949,0.000,-28.655,0.000,302.836,11.306,127.253,308.558,0.000,295.604,1.0,0.000,286.162,0.0,1.256620e+08,76.64,9.630736e+09,3349,ZS,2019-07-31,2019-06-20,0.2200,0.17,1,33037,ZS
1446,44317,2019-12-31,0001677250,028423,ZTO,2019.0,814.899,100.308,6590.620,182.494,2886.435,2353.960,959.492,42.902,814.899,11.528,3175.347,181.612,781.947,5500.875,608.090,1075.270,1.0,72.447,1597.073,0.0,5.818200e+08,19.12,1.112440e+10,3477,ZTO,2019-12-31,2019-06-20,6.7500,6.47,1,28423,ZTO
1447,91122,2020-01-31,0001318008,162988,ZUMZ,2019.0,66.881,0.000,914.258,0.000,412.588,251.196,159.710,61.800,66.881,4.686,1034.129,25.134,25.828,466.086,1.711,448.172,1.0,284.717,198.768,0.0,2.574100e+07,26.10,6.718401e+08,3821,ZUMZ,2020-01-31,2019-06-20,2.6200,1.90,1,162988,ZUMZ
1448,53913,2020-01-31,0001423774,033232,ZUO,2019.0,-83.394,0.818,402.227,0.000,266.784,171.937,165.620,10.187,-83.394,0.432,274.757,11.276,114.482,164.659,0.000,237.568,1.0,68.401,117.662,0.0,8.647100e+07,15.32,1.324736e+09,3711,ZUO,2020-01-31,2019-06-20,-0.3400,-0.42,1,33232,ZUO


## ADV preparation

In [9]:
# A function taking care of all financial values calculation
def  calValues(db, company_list):
    print('Estimated waiting time 4mins, please wait')
    # Prepare dataset for nine models
    # HVZ one year
    df_HVZ_one = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_1'])
    # HVZ one year
    df_HVZ_one_additional_feature = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq', 'E_1'])
    # HVZ two year
    df_HVZ_two = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_2'])
    # HVZ three year
    df_HVZ_three = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_3'])

    # EP one year
    df_EP_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_1'])
    # EP two year
    df_EP_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_2'])
    # EP three year
    df_EP_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_3'])

    # RI one year
    df_RI_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_1'])
    # RI two year
    df_RI_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_2'])
    # RI three year
    df_RI_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_3'])

    X_pred_HVZs = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC'])
    # X_pred_HVZs_add_2019 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'E_1', 'tic', 'cik', 'fyear','mrk_eq'])
    # X_pred_HVZs_add_2022 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'E_1', 'tic', 'cik', 'fyear','mrk_eq'])
    X_pred_EPs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E'])
    X_pred_RIs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC'])

    Y_pred_one = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_two = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_three = pd.DataFrame(columns=['tic','E_deflat'])
    count = 0
    # Consistent values calculation
    # Main model
    # HVZ model
    inforaw['E'] = inforaw['ib'] - inforaw['spi']
    inforaw['DD'] = (inforaw['dvc'] > 0).astype(int)
    inforaw['NegE'] = (inforaw['E'] < 0).astype(int)
    inforaw['AC'] = (inforaw['act'] - inforaw['che']).diff() - (inforaw['lct'] - inforaw['dlc'] - inforaw['txp']).diff() - inforaw['dp']

    # EP model
    inforaw['E_deflat'] = inforaw['E'] / inforaw['csho']
    inforaw['NegE_E'] = inforaw['E_deflat'] * inforaw['NegE']

    # RI model
    inforaw['B'] = inforaw['ceq'] / inforaw['csho']
    inforaw['WC'] = (inforaw['act'] - inforaw['che']) - (inforaw['lct'] - inforaw['dlc'])
    inforaw['NCO'] = (inforaw['at'] - inforaw['act'] - inforaw['ivao']) - (inforaw['lt'] - inforaw['lct'] - inforaw['dltt'])
    inforaw['FIN'] = ((inforaw['ivst'] - inforaw['ivao']) - (inforaw['dltt'] - inforaw['dlc'] - inforaw['pstk'])) 
    inforaw['TACC'] = (inforaw['WC'] + inforaw['NCO'] + inforaw['FIN']) / inforaw['csho']

    # Prediction
    inforaw['E_lag1'] = inforaw.groupby('tic')['E'].shift(-1)
    inforaw['E_lag2'] = inforaw.groupby('tic')['E'].shift(-2)
    inforaw['E_lag3'] = inforaw.groupby('tic')['E'].shift(-3)
    # Additional Features
    # Calculate Shareholder's Equity
    inforaw['shareholders_equity'] = inforaw['at'] - inforaw['lt']

    # Calculate current ratio
    inforaw['current_ratio'] = inforaw['act'] / inforaw['lct']

    # Calculate debt to equity
    inforaw['debt_to_equity'] = inforaw['lt'] / inforaw['shareholders_equity']

    # Calculate return on equity (ROE)
    inforaw['roe'] = inforaw['ni'] / inforaw['shareholders_equity']

    # Calculate Asset Turnover
    inforaw['asset_turnover'] = inforaw['revt'] / inforaw['at']
    X_pred_2019 = inforaw.loc[inforaw['fyear'] == 2019].reset_index(drop=True)
    X_pred_2019 = X_pred_2019.replace([np.inf, -np.inf], np.nan)
    X_pred_2019 = X_pred_2019.dropna().reset_index()
    X_pred_2022 = inforaw.loc[inforaw['fyear'] == 2022].reset_index(drop=True)
    X_pred_2022 = X_pred_2022.replace([np.inf, -np.inf], np.nan)
    X_pred_2022 = X_pred_2022.dropna().reset_index()
    
    for cik in company_list['cik'].astype(str):
        count += 1
        print(count)
        # accrualsinfo = db.raw_sql(dataquery)
        infocomp = inforaw.loc[(inforaw['cik'] == cik)].reset_index(drop=True)
        infocomp = infocomp.sort_values(by='fyear')

        # Compile dataset
        # Drop year differnece axillary 
        infocomp = infocomp[infocomp['fyear'] != 2009]
        # Year range of training
        infocomptrain = infocomp[(infocomp['fyear'] >= 2010) & (infocomp['fyear'] <= 2019)].reset_index(drop=True)
        # Prediction data
        
        # X_pred_2019 = X_pred_2019.dropna()
        # X_pred_2022 = X_pred_2022.dropna()
        if X_pred_2019.empty is not None: 
            # Yearly data
            Y_pred_2020 = infocomp[(infocomp['fyear'] == 2020)].reset_index(drop=True)
            Y_pred_2020 = Y_pred_2020[['tic','E_deflat']] #  !!!!!
            Y_pred_one = Y_pred_one.append(Y_pred_2020, ignore_index=True)

            Y_pred_2021 = infocomp[(infocomp['fyear'] == 2021)].reset_index(drop=True)
            Y_pred_2021 = Y_pred_2021[['tic','E_deflat']]
            Y_pred_two = Y_pred_two.append(Y_pred_2021, ignore_index=True)

            Y_pred_2022 = infocomp[(infocomp['fyear'] == 2022)].reset_index(drop=True)
            Y_pred_2022 = Y_pred_2022[['tic','E_deflat']]
            Y_pred_three = Y_pred_three.append(Y_pred_2022, ignore_index=True)

        ## Calculate HVZ Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        X_additional = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E']].reset_index(drop=True)
        
        X_pred_HVZ = X_pred_2019[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_HVZs = X_pred_HVZs.append(X_pred_HVZ, ignore_index=True)

        y = y.rename(columns={"E": "E_1"})
        HVZ_one = pd.concat([X, y], axis=1)
        df_HVZ_one = df_HVZ_one.append(HVZ_one, ignore_index=True)
        df_HVZ_one = df_HVZ_one.dropna()
        HVZ_one_additional = pd.concat([X_additional, y], axis=1)
        df_HVZ_one_additional_feature = df_HVZ_one_additional_feature.append(HVZ_one_additional, ignore_index=True)
        df_HVZ_one_additional_feature = df_HVZ_one_additional_feature.dropna()

        ## Calculate HVZ Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_2"})
        HVZ_two = pd.concat([X, y], axis=1)
        df_HVZ_two = df_HVZ_two.append(HVZ_two, ignore_index=True)
        df_HVZ_two = df_HVZ_two.dropna()

        ## Calculate HVZ Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_3"})
        HVZ_three = pd.concat([X, y], axis=1)
        df_HVZ_three = df_HVZ_three.append(HVZ_three, ignore_index=True)
        df_HVZ_three = df_HVZ_three.dropna()

        ## Calculate EP Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)

        X_pred_EP = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_EPs = X_pred_EPs.append(X_pred_EP, ignore_index=True)

        y = y.rename(columns={"E_deflat": "E_1"})
        EP_one = pd.concat([X, y], axis=1)
        df_EP_one = df_EP_one.append(EP_one, ignore_index=True)
        df_EP_one = df_EP_one.dropna()

        ## Calculate EP Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        EP_two = pd.concat([X, y], axis=1)
        df_EP_two = df_EP_two.append(EP_two, ignore_index=True)
        df_EP_two = df_EP_two.dropna()

        ## Calculate EP Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        EP_three = pd.concat([X, y], axis=1)
        df_EP_three = df_EP_three.append(EP_three, ignore_index=True)
        df_EP_three = df_EP_three.dropna()

        ## Calculate RI Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)
        
        X_pred_RI = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_RIs = X_pred_RIs.append(X_pred_RI, ignore_index=True)
        
        y = y.rename(columns={"E_deflat": "E_1"})
        RI_one = pd.concat([X, y], axis=1)
        df_RI_one = df_RI_one.append(RI_one, ignore_index=True)
        df_RI_one = df_RI_one.dropna()

        ## Calculate RI Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        RI_two = pd.concat([X, y], axis=1)
        df_RI_two = df_RI_two.append(RI_two, ignore_index=True)
        df_RI_two = df_RI_two.dropna()

        ## Calculate RI Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        RI_three = pd.concat([X, y], axis=1)
        df_RI_three = df_RI_three.append(RI_three, ignore_index=True)
        df_RI_three = df_RI_three.dropna()
        
    return df_HVZ_one, df_HVZ_two, df_HVZ_three, df_HVZ_one_additional_feature,\
        df_EP_one, df_EP_two, df_EP_three, \
            df_RI_one, df_RI_two,df_RI_three, \
                X_pred_HVZs, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three,\
                    X_pred_2019, X_pred_2022
                      

In [10]:
#Compile companylist for companies that EXISTED in EVERY year between startyeartrain to endyeartest to not Nan the value
year = startYear
query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
company_list = db.raw_sql(query_comp) 
for year in np.arange(startYear,endYear):
    query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
    comp_list = db.raw_sql(query_comp) 
    company_list = pd.merge(comp_list,company_list, on= 'cik', how='inner')
company_list = company_list.dropna().reset_index(drop=True)

# Company list obtained, Get company tic for reviewing convenience
query_comp = (
        f"""
        SELECT DISTINCT cik, MIN (tic) AS tic
        FROM comp.funda
        WHERE fyear BETWEEN '{startYear}' AND '{endYear}'
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        group by cik
        """) 
company_tic_list = db.raw_sql(query_comp) 
company_list = pd.merge(company_list, company_tic_list, left_on = 'cik', right_on = 'cik', how='left')
# Make sure we have the tic exist in linkdf so we can find analyst data
company_list = pd.merge(company_list, linkdf, left_on = 'tic', right_on = 'TICKER', how='inner')

print(company_list)
company_list.to_csv('company_list.csv', index=False)
# company_list = company_list.sample(n=500, random_state=42)

             cik   tic   gvkey TICKER
0     0001119774  CGEN    3172   CGEN
1     0001115837   MBT  137433    MBT
2     0000075362  PCAR    8253   PCAR
3     0001395942   KAR  183581    KAR
4     0000871763   MAN    7007    MAN
...          ...   ...     ...    ...
1854  0001289848  HURN  161997   HURN
1855  0001084048    ZD   66716     ZD
1856  0001345105   CPA   21381    CPA
1857  0000882361  APTO   15490   APTO
1858  0001046025  HFWA   66285   HFWA

[1859 rows x 4 columns]


In [11]:
# Data extraction
df_HVZ_one, df_HVZ_two, df_HVZ_three, df_HVZ_one_additional_feature,\
        df_EP_one, df_EP_two, df_EP_three, \
            df_RI_one, df_RI_two,df_RI_three, \
                X_pred_HVZs, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three,\
                    X_pred_2019, X_pred_2022 = calValues(db, company_list)

Estimated waiting time 4mins, please wait
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
26

In [12]:
print(df_HVZ_one_additional_feature)

            at  dvc DD       E NegE     AC  current_ratio  debt_to_equity  \
0       36.458  0.0  0  -7.203    1 -4.113      11.502253        0.288952   
1       29.081  0.0  0 -10.704    1 -4.468      13.479203        0.485164   
2       28.909  0.0  0 -13.628    1  0.168      18.476156        0.635865   
3       56.711  0.0  0 -14.083    1 -5.684       6.876341        0.778443   
4      114.986  0.0  0 -10.302    1 -0.381      11.397481        0.083588   
...        ...  ... ..     ...  ...    ...            ...             ...   
12148   31.600  0.0  0 -11.992    1  0.344      13.487973        0.079530   
12149   21.249  0.0  0 -14.626    1 -0.322       8.834890        0.124702   
12150   11.610  0.0  0 -18.627    1  0.028       6.398305        0.179878   
12151   11.967  0.0  0 -11.661    1 -0.348       6.699717        0.173005   
12152   16.870  0.0  0 -23.868    1 -0.760       5.911079        0.198068   

            roe  asset_turnover   tic         cik   fyear       mrk_eq  \
0

In [13]:
# How to use the object

# 1. Load the dictionary as dataframe
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# 2. Create a TextSentimentAnalyser object
text_analyzer = TextSentimentAnalyser(lm_word_list_df)
  
# 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
# that year it will return null
sentiment = text_analyzer.get_sentiment('0000055772', 2010)
print(sentiment)


Negative


In [14]:
# How to use the object

# 1. Load the dictionary as dataframe
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# 2. Create a TextSentimentAnalyser object
text_analyzer = TextSentimentAnalyser(lm_word_list_df)
df_HVZ_one_additional_feature['sentiment'] = None
for index,row in df_HVZ_one_additional_feature.iterrows():    
    # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
    # that year it will return null
    print(int(row['cik']))
    sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
    df_HVZ_one_additional_feature.at[index, 'sentiment'] = sentiment
    print(sentiment)
df_HVZ_one_additional_feature.to_csv('df_HVZ_one_additional_feature.csv', index=False)

1119774
No 10k found for 0001119774 in 2010.
NoSentiment
1119774
No 10k found for 0001119774 in 2011.
NoSentiment
1119774
No 10k found for 0001119774 in 2012.
NoSentiment
1119774
No 10k found for 0001119774 in 2013.
NoSentiment
1119774
No 10k found for 0001119774 in 2014.
NoSentiment
1119774
No 10k found for 0001119774 in 2015.
NoSentiment
1119774
No 10k found for 0001119774 in 2016.
NoSentiment
1119774
No 10k found for 0001119774 in 2017.
NoSentiment
1119774
No 10k found for 0001119774 in 2018.
NoSentiment
1115837
No 10k found for 0001115837 in 2010.
NoSentiment
1115837
No 10k found for 0001115837 in 2011.
NoSentiment
1115837
No 10k found for 0001115837 in 2012.
NoSentiment
1115837
No 10k found for 0001115837 in 2013.
NoSentiment
1115837
No 10k found for 0001115837 in 2014.
NoSentiment
1115837
No 10k found for 0001115837 in 2015.
NoSentiment
1115837
No 10k found for 0001115837 in 2016.
NoSentiment
1115837
No 10k found for 0001115837 in 2017.
NoSentiment
1115837
No 10k found for 000111

IndexError: list index out of range

In [ ]:
df_HVZ_one_additional_feature

,at,dvc,DD,E,NegE,AC,current_ratio,debt_to_equity,roe,asset_turnover,tic,cik,fyear,mrk_eq,E_1,csho,sentiment
0,253.012,0.000,0,7.986,0,-0.299,2.110224,3.066017,0.128339,0.387219,BBGI,0001099160,2010.0,2.943428e+07,10.100,22.686,Negative
1,254.989,0.000,0,10.100,0,-2.591,2.315038,2.462313,0.137141,0.383150,BBGI,0001099160,2011.0,2.570205e+07,14.366,22.762,Negative
2,259.373,1.928,1,14.366,0,-2.531,2.642529,2.123132,0.132825,0.386474,BBGI,0001099160,2012.0,3.621172e+07,13.807,22.808,NoSentiment
3,264.209,1.024,1,13.807,0,-3.328,2.504331,1.821962,0.123320,0.397057,BBGI,0001099160,2013.0,5.187220e+07,2.717,22.948,Negative
4,315.967,4.108,1,2.717,0,-0.861,2.533542,1.420424,0.306407,0.185798,BBGI,0001099160,2014.0,4.088765e+07,10.392,23.108,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3303,4414.300,195.200,1,445.600,0,-63.800,1.262210,1.439649,0.243396,0.961240,MKC,0000063754,2014.0,8.421418e+09,471.200,256.800,None
3304,4507.800,208.200,1,471.200,0,-187.900,1.134091,1.672239,0.238366,0.953081,MKC,0000063754,2015.0,9.375305e+09,492.500,254.600,None
3305,4635.900,222.000,1,492.500,0,-222.400,0.999367,1.830047,0.287528,0.951595,MKC,0000063754,2016.0,1.229617e+10,579.600,250.600,None
3306,10385.800,247.000,1,579.600,0,-323.800,0.830381,3.039753,0.186316,0.465453,MKC,0000063754,2017.0,1.105656e+10,975.500,262.000,None


In [ ]:
# df_HVZ_one_additional_feature.dropna()
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_HVZ_one_additional_feature['sentiment_encoded'] = encoder.fit_transform(df_HVZ_one_additional_feature['sentiment'])


In [ ]:

df_HVZ_one_additional_feature_test = df_HVZ_one_additional_feature.drop('sentiment', axis=1)
df_HVZ_one_additional_feature_test

,at,dvc,DD,E,NegE,AC,current_ratio,debt_to_equity,roe,asset_turnover,tic,cik,fyear,mrk_eq,E_1,csho,sentiment_encoded
0,253.012,0.000,0,7.986,0,-0.299,2.110224,3.066017,0.128339,0.387219,BBGI,0001099160,2010.0,2.943428e+07,10.100,22.686,1
1,254.989,0.000,0,10.100,0,-2.591,2.315038,2.462313,0.137141,0.383150,BBGI,0001099160,2011.0,2.570205e+07,14.366,22.762,1
2,259.373,1.928,1,14.366,0,-2.531,2.642529,2.123132,0.132825,0.386474,BBGI,0001099160,2012.0,3.621172e+07,13.807,22.808,2
3,264.209,1.024,1,13.807,0,-3.328,2.504331,1.821962,0.123320,0.397057,BBGI,0001099160,2013.0,5.187220e+07,2.717,22.948,1
4,315.967,4.108,1,2.717,0,-0.861,2.533542,1.420424,0.306407,0.185798,BBGI,0001099160,2014.0,4.088765e+07,10.392,23.108,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3303,4414.300,195.200,1,445.600,0,-63.800,1.262210,1.439649,0.243396,0.961240,MKC,0000063754,2014.0,8.421418e+09,471.200,256.800,4
3304,4507.800,208.200,1,471.200,0,-187.900,1.134091,1.672239,0.238366,0.953081,MKC,0000063754,2015.0,9.375305e+09,492.500,254.600,4
3305,4635.900,222.000,1,492.500,0,-222.400,0.999367,1.830047,0.287528,0.951595,MKC,0000063754,2016.0,1.229617e+10,579.600,250.600,4
3306,10385.800,247.000,1,579.600,0,-323.800,0.830381,3.039753,0.186316,0.465453,MKC,0000063754,2017.0,1.105656e+10,975.500,262.000,4


In [ ]:
df_HVZ_one_additional_feature[df_HVZ_one_additional_feature['cik'] == '0000882291']


,at,dvc,DD,E,NegE,AC,current_ratio,debt_to_equity,roe,asset_turnover,tic,cik,fyear,mrk_eq,E_1,csho,sentiment
250,0.567,0.0,0,-2.405,1,-0.773,0.062662,-1.094880,0.955656,0.000000,AEMD,0000882291,2010.0,17448140.0,-7.674,0.103289,None
251,0.839,0.0,0,-7.674,1,-2.154,0.068673,-1.090254,0.872526,1.619785,AEMD,0000882291,2011.0,6843705.0,-4.752,0.156688,None
252,0.497,0.0,0,-4.752,1,-0.799,0.037859,-1.054353,0.534996,2.474849,AEMD,0000882291,2012.0,11188635.0,-11.952,0.231565,None
253,1.695,0.0,0,-11.952,1,-6.971,0.094511,-1.115071,0.906789,0.958112,AEMD,0000882291,2013.0,16681998.0,-4.031,0.299965,None
254,1.381,0.0,0,-4.031,1,13.399,2.097561,1.121352,-10.440860,0.551774,AEMD,0000882291,2014.0,37921310.0,-4.872,0.443802,None
255,2.557,0.0,0,-4.872,1,-0.060,4.561670,0.702397,-3.243675,0.346891,AEMD,0000882291,2015.0,49541100.0,-6.718,0.508132,None
256,1.726,0.0,0,-6.718,1,-0.359,2.609477,1.900840,-12.228571,0.227115,AEMD,0000882291,2016.0,43445400.0,-5.303,0.586465,None
257,7.352,0.0,0,-5.303,1,0.316,15.125523,0.218631,-0.941488,0.020403,AEMD,0000882291,2017.0,18981800.0,-5.722,1.182664,None
258,4.123,0.0,0,-5.722,1,-0.461,2.213816,0.793388,-2.694650,0.055785,AEMD,0000882291,2018.0,22734080.0,-5.927,1.266930,None


In [ ]:
# How to use the object
# 1. Load the dictionary as dataframe
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# 2. Create a TextSentimentAnalyser object
text_analyzer = TextSentimentAnalyser(lm_word_list_df)
X_pred_2019['sentiment'] = None
for index,row in X_pred_2019.iterrows():    
    # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
    # that year it will return null
    print(int(row['cik']))
    sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
    X_pred_2019.at[index, 'sentiment'] = sentiment
    print(sentiment)


1090872
Uncertainty
1675149
Negative
6201
Negative
1158114
Negative
824142
Uncertainty
1158449
Negative
320193
Negative
1713210
Negative
1015647
No 10k found for 0001015647 in 2019.
NoSentiment
1468079
No 10k found for 0001468079 in 2019.
NoSentiment
1135185
Negative
1091587
No 10k found for 0001091587 in 2019.
NoSentiment
1551152
Negative
1140859
Negative
318306
Negative
1565025
No 10k found for 0001565025 in 2019.
NoSentiment
1144980
Negative
907654
Negative
771497
Negative
896747
Negative
1605331
Negative
1800
Uncertainty
1447028
Negative
1739445
Negative
1070494
Negative
1508348
Negative
1683541
No 10k found for 0001683541 in 2019.
NoSentiment
712034
Negative
1002819
No 10k found for 0001002819 in 2019.
NoSentiment
1069308
Negative
880984
Negative
861838
Negative
1520697
Negative
949858
Negative
1651625
No 10k found for 0001651625 in 2019.
NoSentiment
935036
Uncertainty
1261317
No 10k found for 0001261317 in 2019.
NoSentiment
1113232
Uncertainty
868857
Negative
1680062
Negative
146

KeyboardInterrupt: 

In [ ]:
# How to use the object
# 1. Load the dictionary as dataframe
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# 2. Create a TextSentimentAnalyser object
text_analyzer = TextSentimentAnalyser(lm_word_list_df)
X_pred_2022['sentiment'] = None
for index,row in X_pred_2022.iterrows():    
    # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
    # that year it will return null
    print(int(row['cik']))
    sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
    X_pred_2022.at[index, 'sentiment'] = sentiment
    print(sentiment)

## Build Model

### Q1 model

In [ ]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one, 'HVZ_two': df_HVZ_two, 'HVZ_three': df_HVZ_three,
        'EP_one': df_EP_one, 'EP_two': df_EP_two, 'EP_three': df_EP_three,
        'RI_one': df_RI_one, 'RI_two': df_RI_two, 'RI_three': df_RI_three}

for name, df in data.items(): 
    print(name)
    if 'HVZ' in name:
        X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    if 'one' in name:
        y = df['E_1']
    elif 'two' in name:
        y = df['E_2']
    elif 'three' in name:
        y = df['E_3']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8406585034002997
HVZ_two
Adjusted R-squared Score:  0.765161741525018
HVZ_three
Adjusted R-squared Score:  0.7207776428117876
EP_one
Adjusted R-squared Score:  0.6517038844402605
EP_two
Adjusted R-squared Score:  0.37627104332115535
EP_three
Adjusted R-squared Score:  0.09523489577762567
RI_one
Adjusted R-squared Score:  0.6586419773745988
RI_two
Adjusted R-squared Score:  0.527852990597225
RI_three
Adjusted R-squared Score:  0.14284848752340373


In [ ]:
# Compare to analyst 
# One, two and three year models
models = {'HVZ_one': models['HVZ_one'], 'EP_one': models['EP_one'], 'RI_one': models['RI_one'],
          'HVZ_two': models['HVZ_two'], 'EP_two': models['EP_two'], 'RI_two': models['RI_two'],
          'HVZ_three': models['HVZ_three'], 'EP_three': models['EP_three'], 'RI_three': models['RI_three']}


for model_name, model in models.items():
    print(model_name)
    if 'HVZ' in model_name:
        X = X_pred_HVZs[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in model_name:
        X = X_pred_EPs[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in model_name:
        X = X_pred_RIs[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    X = add_constant(X)
    
    if 'one' in model_name:
        y = Y_pred_one.copy()
        print(1)
    elif 'two' in model_name:
        y = Y_pred_two.copy()
        print(2)
    elif 'three' in model_name:
        y = Y_pred_three.copy()
        print(3)
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    # y_pred_df = y_pred_df.
    if 'HVZ' in model_name:
        # y_pred_adjusted = y_pred_df['y_pred'] / X_pred_HVZs['csho']
        y_pred_df['y_pred'] = y_pred_df['y_pred'] / df['csho']
        # y_pred_adjusted_dict[name] = y_pred_adjusted
        # print(y_pred)
    y_pred_df = pd.concat([X_pred_HVZs['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred'])

    print(y_pred_df)



HVZ_one
1
       tic    y_pred  E_deflat      bias
0     AEMD -7.934610 -0.649082  7.285528
1     AEMD -7.934610 -0.649082  7.285528
2     AEMD -7.934610 -0.649082  7.285528
3     AEMD -7.934610 -0.649082  7.285528
4     AEMD -7.934610 -0.649082  7.285528
...    ...       ...       ...       ...
8095  ZUMZ  6.575269  3.165358 -3.409911
8096  ZUMZ  6.575269  3.165358 -3.409911
8097  ZUMZ  6.575269  3.165358 -3.409911
8098  ZUMZ  6.575269  3.165358 -3.409911
8099  ZUMZ  6.575269  3.165358 -3.409911

[8100 rows x 4 columns]
EP_one
1
       tic    y_pred  E_deflat      bias
0     AEMD -1.033327 -0.649082  0.384245
1     AEMD -1.033327 -0.649082  0.384245
2     AEMD -1.033327 -0.649082  0.384245
3     AEMD -1.033327 -0.649082  0.384245
4     AEMD -1.033327 -0.649082  0.384245
...    ...       ...       ...       ...
8095  ZUMZ  2.619489  3.165358  0.545869
8096  ZUMZ  2.619489  3.165358  0.545869
8097  ZUMZ  2.619489  3.165358  0.545869
8098  ZUMZ  2.619489  3.165358  0.545869
8099  ZUMZ  2

In [ ]:
y_pred_df['E_deflat'].describe(include='all')

count     1162.000000
unique    1162.000000
top         -0.388964
freq         1.000000
Name: E_deflat, dtype: float64

In [ ]:
y_pred_df['E_deflat'].max()

127.03147547840634

### Q2 Model, with additional features


In [ ]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one_additional_feature_test}

for name, df in data.items(): 
    print(name)
    X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'sentiment_encoded']]
    y = df['E_1']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    # if 'HVZ' in name:
    #     y_pred_adjusted = y_pred / df['csho']
    #     y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8624354849521335


In [ ]:
# Compare to analyst 
# One, two and three year models
models = {'HVZ_one': models['HVZ_one']}


for model_name, model in models.items():
    print(model_name)
    X_pred_2019 = X_pred_2019.dropna()
    X = X_pred_2019[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
    
    X = add_constant(X)
    
    y = X_pred_2019[['tic', 'E_lag1']]
  
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs['csho']

    # # y_pred_df = y_pred_df.
    # if 'HVZ' in model_name:
    #     # y_pred_adjusted = y_pred_df['y_pred'] / X_pred_HVZs['csho']
        
    #     # y_pred_adjusted_dict[name] = y_pred_adjusted
    #     # print(y_pred)
    y_pred_df = pd.concat([X_pred_2019['tic'], y_pred_df], axis=1)

    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')
    print(y_pred_df)

    y_pred_df['bias'] = (X_pred_2019['E_deflat'] - y_pred_df['y_pred'])/ (X_pred_2019['mrk_eq']/X_pred_2019['csho']/1000000)

    print(y_pred_df)

HVZ_one
       tic     y_pred    E_lag1
0        A   2.183941   885.000
1       AA  -1.410478  -190.000
2      AAL  -0.095256 -9851.000
3     AAOI  -0.844575   -50.677
4     AAON   0.005622    72.609
...    ...        ...       ...
3275  ZVRA  -3.011156   -11.932
3276   ZWS  11.339927   133.600
3277  ZYME  -1.532368  -183.583
3278  ZYNE  -3.922117   -51.337
3279  ZYXI   7.649276     9.151

[3280 rows x 3 columns]
       tic     y_pred    E_lag1      bias
0        A   2.183941   885.000  0.021149
1       AA  -1.410478  -190.000  0.049675
2      AAL  -0.095256 -9851.000  0.149095
3     AAOI  -0.844575   -50.677 -0.185426
4     AAON   0.005622    72.609  0.020427
...    ...        ...       ...       ...
3275  ZVRA  -3.011156   -11.932 -0.354267
3276   ZWS  11.339927   133.600 -0.364156
3277  ZYME  -1.532368  -183.583 -0.097607
3278  ZYNE  -3.922117   -51.337  0.203428
3279  ZYXI   7.649276     9.151 -0.917511

[3280 rows x 4 columns]


In [ ]:
# using 2022 to predict 2023
# Compare to analyst 
# One, two and three year models
models = {'HVZ_one': models['HVZ_one']}


for model_name, model in models.items():
    print(model_name)
    X_pred_2022 = X_pred_2022.dropna()
    X = X_pred_2022[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
    
    X = add_constant(X)
    
    y = X_pred_2022[['tic', 'E_lag1']]
  
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs['csho']

    # # y_pred_df = y_pred_df.
    # if 'HVZ' in model_name:
    #     # y_pred_adjusted = y_pred_df['y_pred'] / X_pred_HVZs['csho']
        
    #     # y_pred_adjusted_dict[name] = y_pred_adjusted
    #     # print(y_pred)
    y_pred_df = pd.concat([X_pred_2022['tic'], y_pred_df], axis=1)

    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')
    print(y_pred_df)

    y_pred_df['bias'] = (X_pred_2022['E_deflat'] - y_pred_df['y_pred'])/ (X_pred_2022['mrk_eq']/X_pred_2022['csho']/1000000)

    print(y_pred_df)

HVZ_one


ValueError: Found array with 0 sample(s) (shape=(0, 11)) while a minimum of 1 is required.

In [ ]:
y_pred_df['y_pred']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
85    NaN
172   NaN
311   NaN
576   NaN
635   NaN
Name: y_pred, Length: 75, dtype: float64

In [ ]:
# # df_analyst_one, y_pred_df
# all_prediction = pd.merge(y_pred_df,df_analyst_three, left_on= 'tic', right_on='ticker', how='inner')
# all_prediction['medest'] = all_prediction['medest']

,tic,y_pred,E_deflat,index,ticker,predict_year,fpedats,statpers,actual,medest,gvkey,TICKER
0,CENX,-0.137355,1.673454,1272,CENX,2021.0,2021-12-31,2019-06-20,-0.62,1.27,62516,CENX
1,JKS,-0.502803,1.532952,498,JKS,2021.0,2021-12-31,2019-06-20,1.84,1.13,184182,JKS
2,D,4.874187,1.329341,398,D,2021.0,2021-12-31,2019-06-20,3.86,4.66,4029,D
3,MTD,17.093470,39.88247,2178,MTD,2021.0,2021-12-31,2019-06-20,34.01,28.50,65772,MTD
4,AZPN,3.400586,0.884936,1483,AZPN,2021.0,2021-06-30,2019-06-20,5.20,4.08,30870,AZPN
...,...,...,...,...,...,...,...,...,...,...,...,...
552,DRRX,-0.254264,-1.550713,1311,DRRX,2021.0,2021-12-31,2019-06-20,-1.60,-1.00,140044,DRRX
553,FSLR,3.783309,-3.075519,1571,FSLR,2021.0,2021-12-31,2019-06-20,4.38,4.01,175404,FSLR
554,PEP,4.619980,7.042847,2223,PEP,2021.0,2021-12-31,2019-06-20,6.26,6.44,8479,PEP
555,CDXS,-0.167413,-0.462309,144,CDXS,2021.0,2021-12-31,2019-06-20,-0.33,0.20,179841,CDXS
